<a href="https://colab.research.google.com/github/GeorgeSherif/NLP-ChatEGP/blob/main/Testing_Using_Arabic_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Testing with the Arabic Dataset.


1.   Load the model.
2.   Load the dataset.
3.   Translate the dataset and preprocess it.
4.   Test using the model.



In [ ]:
!pip install xformers transformers sentencepiece torch pygal torchvision sacremoses


In [4]:
import pandas as pd
import numpy as np
import os
import warnings
import csv
import re
import seaborn as sns
import pygal as py
import torch
import torchvision

from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings("ignore", category=FutureWarning)


In [5]:
from google.colab import drive
from IPython.display import display # Allows the use of display() for DataFrames
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [43]:
from transformers import BertTokenizer,BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = 3,
                                                      id2label={0: 'negative', 1: 'neutral', 2: 'positive'},
                                                      output_attentions = False,
                                                      output_hidden_states = False).to(device)
                                                      
path = F"/content/gdrive/MyDrive/NLP/BERT_ft_epoch3.model"
model.load_state_dict(torch.load(path))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [44]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [11]:
import pandas as pd
import numpy as np
dfTest = pd.read_csv('/content/gdrive/MyDrive/NLP/labelled.csv' ,engine="python")
dfTest

,Sentence,id,label
0,انخفضت أرباح شركة الخليج للمواشي إلى 12،1 مليو...,36,negative
1,تناقش عمومية شركة أرامكس يوم 21 مارس توزيع أرب...,1962,positive
2,حققت العين الأهلية للتأمين أرباحاً بلغت 2 .50 ...,2838,positive
3,ارتفعت أرباح اسمنت أم القيوين بنسبة 30% إلى 7 ...,1599,positive
4,قررت الجمعية العمومية لشركة صناعات أسمنت الفجي...,1528,negative
5,ينعقد يوم الثلاثاء المقبل اجتماع مجلس إدارة بن...,3615,neutral
6,"دبي ""الخليج"":بلغت أرباح شركة ""دبي للتطوير"" في ...",5050,positive
7,أعلنت شركة العين الأهلية للتأمين عن أنها ستفصح...,220,neutral
8,يعتزم بنك إتش إس بي سي إنشاء صندوق بقيمة 100 م...,2101,positive
9,يعقد اجتماع مجلس الإدارة للشركة العربية للصناع...,4559,neutral


In [12]:
from transformers import MarianTokenizer, MarianMTModel
mname = "Helsinki-NLP/opus-mt-tc-big-ar-en"
tokenizer = MarianTokenizer.from_pretrained(mname)
model_translate = MarianMTModel.from_pretrained(mname)

In [13]:
translated = []
for i in range(len(dfTest)):
  input = dfTest['Sentence'][i]
  translated_tokens = model_translate.generate(**tokenizer.prepare_seq2seq_batch([input], return_tensors="pt"))
  translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
  translated.append(translated_text[0])
dfTest['English'] = translated

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [14]:
dfTest = dfTest.drop(["Sentence","id"] , axis =1)
dfTest = dfTest.rename(columns={'label': 'Sentiment', 'English':'Sentence'})
dfTest['Sentiment'] = dfTest['Sentiment'].replace(['negative','neutral','positive'],[0,1,2])
dfTest

,Sentiment,Sentence
0,0,The profits of the Gulf Livestock Company fell...
1,2,Aramex to discuss cash dividend of 16% for 2016
2,2,Al Ain Al Ahlia Insurance achieved a profit of...
3,2,Umm Al Quwain Cement’s profit increased by 30%...
4,0,The General Assembly of Fujairah Cement Indust...
5,1,"Next Tuesday, ADCB’s Board of Directors will m..."
6,2,"Dubai: Last year, Dubai Development Company pr..."
7,1,Al Ain Al Ahlia Insurance Company announced th...
8,2,HSBC to set up $100 million fund to support UA...
9,1,The meeting of the Board of Directors of the A...


In [45]:
#encode Arabic Testing set
encoded_data_val_Arabic = tokenizer.batch_encode_plus( dfTest.Sentence.values,
                                                       add_special_tokens = True,
                                                       return_attention_mask = True,
                                                       return_tensors = 'pt', 
                                                       truncation=True,
                                                       padding=True,
                                                      max_length = 90)

In [46]:
input_ids_val_Arabic = encoded_data_val_Arabic['input_ids']
attention_masks_val_Arabic = encoded_data_val_Arabic['attention_mask']
labels_val_Arabic = torch.tensor(dfTest.Sentiment.values)

In [47]:
from torch.utils.data import TensorDataset

#Arabic Validation set
dataset_val_Arabic = TensorDataset(input_ids_val_Arabic, 
                                   attention_masks_val_Arabic, 
                                   labels_val_Arabic)

In [48]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8



#Arabic validation set
dataloader_val_Arabic = DataLoader(dataset_val_Arabic,
                                   sampler = RandomSampler(dataset_val_Arabic),
                                   batch_size = 8) #since we don't have to do backpropagation for this step

In [49]:
#accuracy score
def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    #make prediction
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        #print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n -> {len(y_preds[y_preds==label]) / len(y_true)}')

In [50]:
def evaluate(dataloader_val):

    #evaluation mode 
    model.eval()
    #tracking variables
    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        #load into GPU
        batch = tuple(b.to(device) for b in batch)

        #define inputs
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]}

        #compute logits
        with torch.no_grad():        
            outputs = model(**inputs)
        
        #compute loss
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        #compute accuracy
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    #compute average loss
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [51]:
from tqdm import tqdm

_,predictions, true_vals = evaluate(dataloader_val_Arabic)

100%|██████████| 5/5 [00:03<00:00,  1.28it/s]


In [52]:
true_vals.shape
accuracy_per_class(predictions, true_vals)

Accuracy:8/9
 -> 0.8888888888888888
Accuracy:12/12
 -> 1.0
Accuracy:9/15
 -> 0.6


In [54]:
# preprocess the sentence
correct = 0
false = 0
positive = 0
negative = 0
neutral = 0
for i in range(len(dfTest)):
  row = dfTest.iloc[i]
  sentence = row['Sentence']
  label = row['Sentiment']
  input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)
  attention_mask = torch.tensor([1] * input_ids.shape[1]).unsqueeze(0)

  # get the predicted sentiment label
  with torch.no_grad():
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      _, predicted_label = torch.max(outputs[0], dim=1)
  print(sentence, predicted_label,label)
  if(label == 0):
    negative = negative + 1
  elif(label == 1):
    neutral = neutral + 1
  elif(label == 2):
    positive = positive + 1
  if(predicted_label == label):
    correct = correct + 1
  else:
    false = false + 1
  print(correct,false)

The profits of the Gulf Livestock Company fell to 12.1 million dirhams in 2010, compared to 27.6 million dirhams in 2009. tensor([0]) 0
1 0
Aramex to discuss cash dividend of 16% for 2016 tensor([1]) 2
1 1
Al Ain Al Ahlia Insurance achieved a profit of AED 2.50 million in 2012 compared to AED 2.53 million in 2011. tensor([0]) 2
1 2
Umm Al Quwain Cement’s profit increased by 30% to AED 7.22 million in the first quarter of this year tensor([2]) 2
2 2
The General Assembly of Fujairah Cement Industries decided not to distribute cash dividends or shares for the year 2010. tensor([1]) 0
2 3
Next Tuesday, ADCB’s Board of Directors will meet to discuss some general matters. tensor([1]) 1
3 3
Dubai: Last year, Dubai Development Company profit reached AED 735,000 compared to AED 9,444,000 in 2013 tensor([2]) 2
4 3
Al Ain Al Ahlia Insurance Company announced that it will disclose its financial results for the third quarter of 2011 on 31/10/2011 tensor([1]) 1
5 3
HSBC to set up $100 million fund t

In [ ]:
print("Accuracy of testing = ", 100 * correct / (correct + false)) 
print(negative,positive,neutral) 